# Trabalho Prático Interpretabilidade e Causalidade - Rafael Antunes (a55336)

#### Imports

In [19]:
from cnn_framework import Modelo_CNN
from rl_class_framework import Modelo_RL_Classifier
import shap
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Importação Modelo Classificador Regressão Logística

In [8]:
modelo_rlclass = Modelo_RL_Classifier()
modelo_rlclass.setup()
modelo_rlclass.show_existing_classes()
print("Precisão", modelo_rlclass.model_accuracy())

A Carregar Dataset
Train: 426 samples
Val: 71 samples
Test: 72 samples
Datast Carregado
Modelo encontrado!
B (Benign) = 1 | M (Malign) = 0
Precisão 0.9583333134651184


In [25]:
X_train_np = modelo_rlclass.train_ds.drop("diagnosis", axis=1).to_numpy()

# Small background for DeepExplainer
background = X_train_np[np.random.choice(X_train_np.shape[0], 50, replace=False)]

# DeepExplainer for Keras model
explainer = shap.DeepExplainer(modelo_rlclass.modelo, background)

# Compute SHAP values
shap_values = explainer.shap_values(X_train_np)  # or a test set

# Pick the first sample
i = 0
sample_shap = shap_values[0][i]
sample_features = X_train_np[i]

feature_names = modelo_rlclass.train_ds.drop("diagnosis", axis=1).columns

# Print contributions
for name, value, shap_val in zip(feature_names, sample_features, sample_shap):
    print(f"{name}: value={value:.3f}  SHAP={shap_val:+.4f}")

radius_mean: value=1.659  SHAP=-0.0015


## Importação do Modelo da CNN

In [2]:
modelo_cnn = Modelo_CNN()
modelo_cnn.setup()

A criar datasets
Found 24997 files belonging to 2 classes.
Using 19998 files for training.


I0000 00:00:1766509901.776992   66312 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6624 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070, pci bus id: 0000:07:00.0, compute capability: 7.5


Found 24997 files belonging to 2 classes.
Using 4999 files for validation.
Shape das Imagens: (32, 128, 128, 3)
Exemplo de Labels: [[1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
  1. 0. 1. 0. 0. 0. 0. 1.]]
Train: 20000
Val: 2496
Test: 2528
Datasets Criados
Modelo encontrado!


In [6]:
print(modelo_cnn.model_accuracy())

0.9880144000053406
